In [1]:
import nltk.translate.gleu_score as gleu
import string
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
!pip install rouge

In [3]:
from rouge import FilesRouge

def get_rouge(hyp_path, ref_path):
    files_rouge = FilesRouge()
    scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
    return scores

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Thiết bị sử dụng: GPU nếu có, ngược lại dùng CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tải mô hình và tokenizer từ Hugging Face
model_name = "HuyTran1301/codeT5-phase1-v2-ep1-full-done"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Đưa mô hình lên thiết bị (GPU/CPU)
model.to(DEVICE)


config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

2025-06-05 04:23:41.679008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749097421.873852      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749097421.931785      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/511k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.29M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.08k [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [5]:
lang = ['C_sharp', 'Python','Java','JS']

In [6]:
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter


def build_bi_gram_vector(predictions):
    bi_gram_predictions = [] 
    bi_grams = []
    for predict in predictions:
        bi_gram_result = list(zip(*[predict[i:] for i in range(2)]))
        bi_grams += bi_gram_result
        bi_gram_predictions.append(bi_gram_result)
    bi_gram_count = Counter(bi_grams) 
    bi_gram_ids = {} 
    for i, bi_gram_key in enumerate(bi_gram_count.keys()):
        bi_gram_ids[bi_gram_key] = i
    prediction_vectors = []
    for p in bi_gram_predictions:
        initial = np.zeros(len(bi_gram_count.keys()))
        for b in p:
            initial[bi_gram_ids[b]] = bi_gram_count[b]
        prediction_vectors.append(initial)
    return np.array(prediction_vectors)


def scoring_prediction_bi_gram(predictions):
    with_weight = []
    bi_gram_predictions = []
    bi_grams = []
    for predict in predictions:
        # print(predict)
        bi_gram_result = list(zip(*[predict[i:] for i in range(2)]))
        # print(bi_gram_result)
        bi_grams += bi_gram_result
        bi_gram_predictions.append(bi_gram_result)
    bi_gram_count = Counter(bi_grams)
    for i in range(len(predictions)):
        weight = sum([bi_gram_count[bg] for bg in bi_gram_predictions[i]])
        with_weight.append((predictions[i], i, weight))
    with_weight_sorted = sorted(with_weight, key=itemgetter(2), reverse=True)
    # print(with_weight_sorted)
    return with_weight_sorted

def scoring_prediction_bi_gram_average(predictions):
    with_weight = []
    bi_gram_predictions = []
    bi_grams = []
    for predict in predictions:
        # print(predict)
        bi_gram_result = list(zip(*[predict[i:] for i in range(2)]))
        # print(bi_gram_result)
        bi_grams += bi_gram_result
        bi_gram_predictions.append(bi_gram_result)
    bi_gram_count = Counter(bi_grams)
    for i in range(len(predictions)):
        weight = sum([bi_gram_count[bg] for bg in bi_gram_predictions[i]])
        weight = weight/len(bi_gram_predictions[i])
        with_weight.append((predictions[i], i, weight))
    with_weight_sorted = sorted(with_weight, key=itemgetter(2), reverse=True)
    # print(with_weight_sorted)
    return with_weight_sorted

# with open('/RQ3/fJS_30_candidate.txt','r',encoding='utf-8') as f, open('m3out30_JS.txt','w',encoding='utf-8') as f1:


In [7]:
num_gen=1
for i in range (len(lang)):
    df = pd.read_csv('/kaggle/input/dataset-merged/test_'+lang[i]+'_merged_data.csv')
    print(lang[i])
    with open(f'{lang[i]}_{num_gen}_candidate.txt', 'w', encoding='utf-8') as f:
        for index, row in df.iterrows():
            inputs = tokenizer(row['text'].strip(), return_tensors="pt", max_length=512, padding="max_length", truncation=True).to(DEVICE)
            # summary_ids = model.generate(inputs["input_ids"], num_beams=j, num_return_sequences = j,  min_length=2, max_length=48, top_p = 0.9, top_k = 5)
            summary_ids = model.generate(inputs["input_ids"], num_beams=num_gen, num_return_sequences = num_gen,  min_length=2, max_length=48, top_p = 0.9, top_k = 5, length_penalty = 0.0)

             
            for k in range(num_gen):
                # summary_ids = model.generate(inputs, num_beams=4, min_length=2, max_length=48, top_p = 0.1)

                best_outputs = tokenizer.decode(summary_ids[k], skip_special_tokens=True, clean_up_tokenization_spaces=False)
                # print(best_outputs)

                f.write(best_outputs+'\n')
            # print(best_outputs)
            # print(count_matching_words(row['text'],best_outputs))
            # f.write(str(count_matching_words(row['text'],best_outputs))+'\n')
            
        

C_sharp


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Python


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Java


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


JS


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [8]:
for type_lang in lang:
    with open('/kaggle/working/'+type_lang+'_'+str(num_gen)+'_candidate.txt','r',encoding='utf-8') as f, open(type_lang+'_'+str(num_gen)+'_ranking.txt','w',encoding='utf-8') as f1:
            i = 0
            ans = []
            ans1 = []
            for line in f:
                i += 1
                x = line.strip().split()
                ans.append(x)
                if i<num_gen: 
                    continue
                else:
                    ans1.append(ans)
                    i = 0
                    ans = []
            for predict in ans1:
                # print(predict)
                scoring_prediction_bi_gram(predict)
        
                # break
                f1.write(' '.join(scoring_prediction_bi_gram(predict)[0][0])+'\n')
                # print(scoring_prediction_bi_gram(predict))

In [9]:
import nltk.translate.gleu_score as gleu
def score_gleu(reference, hypothesis):
    score = 0
    for ref, hyp in zip(reference, hypothesis):
        score += gleu.sentence_gleu([ref.split()], hyp.split())
    return float(score) / len(reference)

In [10]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
def computeBleu1_to_4(reference_list, candidate_list):
    smooth = SmoothingFunction().method1
    bleu1_sum = bleu2_sum = bleu3_sum = bleu4_sum = bleuA_sum = 0

    for (ref, cand) in zip(reference_list, candidate_list):

        tokens_real = ref.split(' ')
        tokens_pred = cand.split(' ')

        if cand == '':
            bleu1_score = bleu2_score = bleu3_score = bleu4_score = bleuA_score = 0

        else:
            bleu1_score = sentence_bleu([tokens_real], tokens_pred, weights=(1.0, 0.0, 0.0, 0.0), smoothing_function=smooth)
            bleu2_score = sentence_bleu([tokens_real], tokens_pred, weights=(0.0, 1.0, 0.0, 0.0), smoothing_function=smooth)
            bleu3_score = sentence_bleu([tokens_real], tokens_pred, weights=(0.0, 0.0, 1.0, 0.0), smoothing_function=smooth)
            bleu4_score = sentence_bleu([tokens_real], tokens_pred, weights=(0.0, 0.0, 0.0, 1.0), smoothing_function=smooth)
            bleuA_score = sentence_bleu([tokens_real], tokens_pred, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth)

        bleu1_sum += bleu1_score
        bleu2_sum += bleu2_score
        bleu3_sum += bleu3_score
        bleu4_sum += bleu4_score
        bleuA_sum += bleuA_score

    return {
    "BLEU_A": round(bleuA_sum / len(reference_list), 3) * 100,
    "BLEU_1": round(bleu1_sum / len(reference_list), 3) * 100,
    "BLEU_2": round(bleu2_sum / len(reference_list), 3) * 100,
    "BLEU_3": round(bleu3_sum / len(reference_list), 3) * 100,
    "BLEU_4": round(bleu4_sum / len(reference_list), 3) * 100,
}

In [11]:
for type_lang in lang:
    df = pd.read_csv("/kaggle/input/dataset-merged/test_"+type_lang+"_merged_data.csv")
    df['title'] = df['title'].fillna("").astype(str)
    df['title'].astype(str).to_csv("target_"+type_lang+".txt", index=False, header=False)

In [12]:
from rouge import FilesRouge

def get_rouge(hyp_path, ref_path):
    files_rouge = FilesRouge()
    scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
    return scores

In [13]:
results = []
for type_lang in lang:
    scores_rouge = get_rouge("/kaggle/working/"+type_lang+"_"+str(num_gen)+"_ranking.txt", "/kaggle/working/target_"+type_lang+".txt")
    with open('/kaggle/working/target_'+type_lang+'.txt','r',encoding='utf-8') as f, open('/kaggle/working/'+type_lang+'_'+str(num_gen)+'_ranking.txt','r',encoding='utf-8') as f1:
        reference = [line.strip() for line in f.readlines()]
        hypothesis = [line.strip() for line in f1.readlines()]
    
    gleu_score = score_gleu(reference, hypothesis)
    bleu = computeBleu1_to_4(reference,hypothesis)
    result = {
    "Language": type_lang,
    "ROUGE-1": round(scores_rouge["rouge-1"]["f"], 3),
    "ROUGE-2": round(scores_rouge["rouge-2"]["f"], 3),
    "ROUGE-L": round(scores_rouge["rouge-l"]["f"], 3),
    "GLEU": round(gleu_score, 3),
    "BLEU_1": round(bleu["BLEU_1"], 3),
    "BLEU_2": round(bleu["BLEU_2"], 3),
    "BLEU_3": round(bleu["BLEU_3"], 3),
    "BLEU_4": round(bleu["BLEU_4"], 3),
    "BLEU_A": round(bleu["BLEU_A"], 3),
    }
    results.append(result)

In [14]:
df_result = pd.DataFrame(results)
df_result
print(df_result)
df_result.to_csv('evaluation_results.csv', index=False)

  Language  ROUGE-1  ROUGE-2  ROUGE-L   GLEU  BLEU_1  BLEU_2  BLEU_3  BLEU_4  \
0  C_sharp    0.229    0.086    0.214  0.078    15.3     6.3     3.5     2.5   
1   Python    0.257    0.085    0.235  0.086    17.6     6.7     3.6     2.6   
2     Java    0.231    0.075    0.213  0.077    15.7     5.9     3.2     2.4   
3       JS    0.244    0.084    0.226  0.082    16.0     6.3     3.6     2.7   

   BLEU_A  
0     4.5  
1     4.8  
2     4.3  
3     4.7  
